<a href="https://colab.research.google.com/github/gitaroktato/deep-learning-exercises/blob/verne-rnn-generator/%20Verne_text_generation_using_RNN..ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation in the style of Jules Verne using RNN


In [1]:
import tensorflow as tf
# import matplotlib.pyplot as plt
import numpy as np
import platform
import time
import pathlib
import os

print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

2024-10-02 16:21:57.355303: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 16:21:57.363545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 16:21:57.373544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 16:21:57.376318: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 16:21:57.385027: I tensorflow/core/platform/cpu_feature_guar

Python version: 3.9.20
Tensorflow version: 2.17.0
Keras version: 3.5.0


## Loading the dataset
Load the dataset and read the data. Take a look in the text.

In [2]:
path_to_file = './docs/utazas_a_holdra.txt'
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 662630 characters


In [3]:
# First few characters of the text
print(text[:250])

JULES VERNE
UTAZÁS A HOLDBA
______
UTAZÁS A HOLD KÖRÜL
KÉT REGÉNY
FORDÍTOTTA: KILÉNYI MÁRIA
2
TARTALOM
UTAZÁS A HOLDBA
1. A GUN CLUB
2. BARBICANE ELNÖK BEJELENTÉSE
3. A BEJELENTÉS HATÁSA
4. A CAMBRIDGE-I CSILLAGVIZSGÁLÓ VÁLASZA
5. A HOLD REGÉNYE
6. A


In [4]:
# The unique characters in the file. Note, that it contains Hungarian character set.
vocab = sorted(set(text))

print('{} unique characters'.format(len(vocab)))
print('vocab:', vocab)

118 unique characters
vocab: ['\n', ' ', '!', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°', 'Á', 'É', 'Í', 'Ó', 'Ö', 'Ú', 'Ü', 'á', 'è', 'é', 'í', 'ó', 'ô', 'ö', 'ú', 'ü', 'Ő', 'ő', 'Ű', 'ű', 'π', '’', '”', '„', '−', '\uf8eb', '\uf8ec', '\uf8ed', '\uf8ee', '\uf8ef', '\uf8f0', '\uf8f6', '\uf8f7', '\uf8f8', '\uf8f9', '\uf8fa', '\uf8fb']


# Process the text


In [5]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

print('{')
for char, _ in zip(vocab, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), ids_from_chars(char)))
print('  ...\n}')

I0000 00:00:1727878934.071622    2007 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727878934.089278    2007 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727878934.089318    2007 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727878934.090805    2007 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727878934.090842    2007 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

{
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '%' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  '*' :   8,
  '+' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  '6' :  20,
  ...
}


In [6]:
# Convert chars in text to indices.
text_as_int = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

print('text_as_int length: {}'.format(len(text_as_int)))
print('{} --> {}'.format(repr(text[:15]), repr(text_as_int[:15].numpy())))

text_as_int length: 662630
'JULES VERNE\nUTA' --> array([37, 48, 39, 32, 46,  2, 49, 32, 45, 41, 32,  1, 48, 47, 28])


## Create training examples and targets

In [7]:
# The maximum length sentence we want for a single input in characters.
sequence_length = 100
examples_per_epoch = len(text) // (sequence_length + 1)

print('examples_per_epoch:', examples_per_epoch)

examples_per_epoch: 6560


In [8]:
# Create training dataset.
ids_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for id in ids_dataset.take(10):
    print(chars_from_ids(id).numpy().decode('utf-8'))

J
U
L
E
S
 
V
E
R
N


2024-10-02 16:22:23.124932: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# Generate batched sequences out of the char_dataset.
sequences = ids_dataset.batch(sequence_length + 1, drop_remainder=True)

# Sequences size is the same as examples_per_epoch.
print('Sequences count: {}'.format(len(list(sequences.as_numpy_iterator()))));
print()

# Sequences examples.
for item in sequences.take(5):
    print(text_from_ids(item).numpy().decode('UTF-8'))

Sequences count: 6560

JULES VERNE
UTAZÁS A HOLDBA
______
UTAZÁS A HOLD KÖRÜL
KÉT REGÉNY
FORDÍTOTTA: KILÉNYI MÁRIA
2
TARTALO
M
UTAZÁS A HOLDBA
1. A GUN CLUB
2. BARBICANE ELNÖK BEJELENTÉSE
3. A BEJELENTÉS HATÁSA
4. A CAMBRIDGE-
I CSILLAGVIZSGÁLÓ VÁLASZA
5. A HOLD REGÉNYE
6. AMIT MINDEN AMERIKAINAK TUDNIA KELL, ÉS AMIT EGYETLEN 
AMERIKAINAK SEM
SZABAD HINNIE TÖBBÉ
7. AZ ÁGYÚGOLYÓ HIMNUSZA
8. AZ ÁGYÚ TÖRTÉNETE
9. A LŐPORKÉRDÉS
10
. HUSZONÖTMILLIÓ BARÁT ÉS EGYETLENEGY ELLENSÉG
11. FLORIDA ÉS TEXAS
12. URBI ET ORBI
13. STONE’S HILL


2024-10-02 16:22:25.382971: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [11]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [12]:
dataset = sequences.map(split_input_target)

# Dataset size is the same as examples_per_epoch.
# But each element of a sequence is now has length of `sequence_length`
# and not `sequence_length + 1`.
print('dataset size: {}'.format(len(list(dataset.as_numpy_iterator()))))

dataset size: 6560


2024-10-02 16:22:33.198635: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [13]:
for input_example, target_example in dataset.take(1):
    print('Input sequence size:', repr(len(input_example.numpy())))
    print('Target sequence size:', repr(len(target_example.numpy())))
    print()
    print('Input:', text_from_ids(input_example).numpy().decode('UTF-8'))
    print('Target:', text_from_ids(target_example).numpy().decode('UTF-8'))

Input sequence size: 100
Target sequence size: 100

Input: JULES VERNE
UTAZÁS A HOLDBA
______
UTAZÁS A HOLD KÖRÜL
KÉT REGÉNY
FORDÍTOTTA: KILÉNYI MÁRIA
2
TARTAL
Target: ULES VERNE
UTAZÁS A HOLDBA
______
UTAZÁS A HOLD KÖRÜL
KÉT REGÉNY
FORDÍTOTTA: KILÉNYI MÁRIA
2
TARTALO


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print('Step {:2d}'.format(i))
    print('  input: {} ({:s})'.format(input_idx, chars_from_ids(input_idx).numpy().decode('UTF-8')))
    print('  expected output: {} ({:s})'.format(target_idx, chars_from_ids(target_idx).numpy().decode('UTF-8')))

Step  0
  input: 37 (J)
  expected output: 48 (U)
Step  1
  input: 48 (U)
  expected output: 39 (L)
Step  2
  input: 39 (L)
  expected output: 32 (E)
Step  3
  input: 32 (E)
  expected output: 46 (S)
Step  4
  input: 46 (S)
  expected output: 2 ( )


## Create training batches

In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [16]:
print('Batched dataset size: {}'.format(len(list(dataset.as_numpy_iterator()))))

Batched dataset size: 102


In [17]:
for input_text, target_text in dataset.take(1):
    print('1st batch: input_text:', input_text)
    print()
    print('1st batch: target_text:', target_text)

1st batch: input_text: tf.Tensor(
[[60 59 73 ... 59 80 80]
 [57 73 91 ... 66 67 55]
 [63 74 59 ... 61 59 73]
 ...
 [ 2 76 63 ... 89 74 69]
 [66 59 73 ... 59 66 74]
 [66 66 93 ... 59 61 92]], shape=(64, 100), dtype=int64)

1st batch: target_text: tf.Tensor(
[[59 73 80 ... 80 80 59]
 [73 91 74 ... 67 55 80]
 [74 59 72 ... 59 73 59]
 ...
 [76 63 73 ... 74 69 67]
 [59 73 80 ... 66 74 59]
 [66 93  1 ... 61 92 74]], shape=(64, 100), dtype=int64)


## Build The Model

In [18]:
# Let's do a quick detour and see how Embeding layer works.
# It takes several char indices sequences (batch) as an input.
# It encodes every character of every sequence to a vector of tmp_embeding_size length.
tmp_vocab_size = 10
tmp_embeding_size = 5
tmp_input_length = 8
tmp_batch_size = 2

tmp_model = tf.keras.models.Sequential()
tmp_model.add(tf.keras.layers.Embedding(
  input_dim=tmp_vocab_size,
  output_dim=tmp_embeding_size,
  input_length=tmp_input_length
))
# The model will take as input an integer matrix of size (batch, input_length).
# The largest integer (i.e. word index) in the input should be no larger than 9 (tmp_vocab_size).
# Now model.output_shape == (None, 10, 64), where None is the batch dimension.
tmp_input_array = np.random.randint(
  low=0,
  high=tmp_vocab_size,
  size=(tmp_batch_size, tmp_input_length)
)
tmp_model.compile('rmsprop', 'mse')
tmp_output_array = tmp_model.predict(tmp_input_array)

print('tmp_input_array shape:', tmp_input_array.shape)
print('tmp_input_array:')
print(tmp_input_array)
print()
print('tmp_output_array shape:', tmp_output_array.shape)
print('tmp_output_array:')
print(tmp_output_array)

/home/gitaroktato/Projects/deep-learning-exercises/.venv/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1727878960.345240    2053 service.cc:146] XLA service 0x7fe300003f80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727878960.345281    2053 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2024-10-02 16:22:40.397325: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
tmp_input_array shape: (2, 8)
tmp_input_array:
[[5 5 7 0 8 0 8 5]
 [7 1 9 6 3 6 9 1]]

tmp_output_array shape: (2, 8, 5)
tmp_output_array:
[[[ 0.00185864 -0.02100599 -0.01260793 -0.02232742 -0.03880433]
  [ 0.00185864 -0.02100599 -0.01260793 -0.02232742 -0.03880433]
  [-0.00946404  0.03090694  0.03300622 -0.0216624   0.01370105]
  [ 0.02836508 -0.033585    0.04161905  0.03932513 -0.03019981]
  [ 0.03100076  0.0426369   0.025365    0.02066356 -0.03179627]
  [ 0.02836508 -0.033585    0.04161905  0.03932513 -0.03019981]
  [ 0.03100076  0.0426369   0.025365    0.02066356 -0.03179627]
  [ 0.00185864 -0.02100599 -0.01260793 -0.02232742 -0.03880433]]

 [[-0.00946404  0.03090694  0.03300622 -0.0216624   0.01370105]
  [ 0.03315396  0.01271433 -0.03646644 -0.00328575 -0.01731677]
  [ 0.02633375  0.0260104  -0.02483853 -0.04419617 -0.00911171]
  [ 0.01703577  0.03164811 -0.03299233  0.02200061  0.00288995]
  [-0.03768346 -0.04272367 -0.03252758 -0.00411167  

I0000 00:00:1727878960.574727    2053 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [19]:
# Length of the vocabulary in chars.
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension.
embedding_dim = 256

# Number of RNN units.
rnn_units = 1024

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Embedding(
      input_dim=vocab_size,
      output_dim=embedding_dim
    ))

    model.add(tf.keras.layers.GRU(
      units=rnn_units,
      return_sequences=True,
      stateful=True,
      recurrent_initializer=tf.keras.initializers.GlorotNormal()
    ))

    model.add(tf.keras.layers.Dense(vocab_size))

    return model

In [21]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

## Try the model

In [22]:
for input_example_batch, target_example_batch in dataset.take(10):
    print(input_example_batch.shape)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)
(64, 100, 119) # (batch_size, sequence_length, vocab_size)


2024-10-02 16:22:50.808795: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [23]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (64, 100, 256)         │        30,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (64, 100, 1024)        │     3,938,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 100, 119)         │       121,975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,090,743 (15.60 MB)

 Trainable params: 4,090,743 (15.60 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
print('Prediction for the 1st letter of the batch 1st sequense:')
print(example_batch_predictions[0, 0])

Prediction for the 1st letter of the batch 1st sequense:
tf.Tensor(
[-1.81233871e-03  2.31324565e-02 -1.37233511e-02  1.03080403e-02
  1.15984511e-02 -2.59139412e-03  1.26301674e-02 -1.04524344e-02
 -7.53531558e-03  1.26383239e-02  1.64030287e-02 -9.93223768e-03
  9.58671153e-04 -8.63767415e-03  7.57475616e-03  5.37779275e-03
 -1.66282412e-02 -6.64332928e-03  1.74230430e-02  5.11807762e-03
  6.18073205e-03 -8.16850085e-03  2.58266646e-02 -8.54787324e-03
  5.87960007e-03  6.12776494e-03  8.73023830e-03  1.17697576e-02
  2.04232987e-02 -4.05841693e-03 -2.16349796e-03 -5.95980091e-03
 -8.68052617e-03 -1.05409522e-03 -2.64295237e-03  2.11315323e-02
  4.58633713e-03  4.06197598e-03  8.13912507e-03  7.68713886e-04
  1.82122719e-02  5.45066921e-03 -2.91244779e-03  6.75075175e-03
  7.70464167e-03  1.25372726e-02 -1.07668969e-03  5.68728568e-03
 -3.44826723e-04 -2.48803594e-03  1.01106362e-02  6.11858908e-04
  3.53057659e-03  7.94468971e-04  1.51895173e-02  1.14990762e-02
  1.32699627e-02  1.21

## Train

In [25]:
# An objective function.
# The function is any callable with the signature scalar_loss = fn(y_true, y_pred).
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )

example_batch_loss = loss(target_example_batch, example_batch_predictions)

print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 119)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.7783422


In [26]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam_optimizer,
    loss=loss
)

## Configure checkpoints

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [28]:
EPOCHS=40

import keras

keras.config.disable_traceback_filtering()
# keras.backend.set_image_data_format("channels_last")

history = model.fit(
  x=dataset,
  epochs=EPOCHS,
  callbacks=[
    checkpoint_callback
  ]
)

Epoch 1/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 3.6209
Epoch 2/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 2.3490
Epoch 3/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 2.0536
Epoch 4/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - loss: 1.8303
Epoch 5/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.6691
Epoch 6/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.5565
Epoch 7/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.4740
Epoch 8/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.4001
Epoch 9/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.3447
Epoch 10/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.2919
Epoch 11/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - loss: 1.2439
Epoch 12/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.1967
Epoch 13/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.1480
Epoch 14/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.1074
Epoch 15/40
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s

In [51]:
# num_generate
# - number of characters to generate.
#
# temperature
# - Low temperatures results in more predictable text.
# - Higher temperatures results in more surprising text.
# - Experiment to find the best setting.
def generate_text(model, start_string, num_generate = 1000, temperature=1.0):
    # Evaluation step (generating text using the learned model)

    # Converting our start string to numbers (vectorizing).
    input_indices = [ids_from_chars(s) for s in start_string]
    input_indices = tf.expand_dims(input_indices, 0)

    # Empty string to store our results.
    text_generated = []

    # Here batch size == 1.
    # model.reset_states()
    for char_index in range(num_generate):
        predictions = model(input_indices)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Using a categorical distribution to predict the character returned by the model.
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
        predictions,
        num_samples=1
        )[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state.
        input_indices = tf.expand_dims([predicted_id], 0)

        text_generated.append(chars_from_ids(predicted_id).numpy().decode('UTF-8'))

    return (start_string + ''.join(text_generated))

In [56]:
tf.train.latest_checkpoint(checkpoint_dir)

In [58]:
simplified_batch_size = 1
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.build(tf.TensorShape([simplified_batch_size, None]))
model.load_weights('./training_checkpoints/ckpt_40.weights.h5')
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ (1, None, 256)         │        30,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_15 (GRU)                    │ (1, None, 1024)        │     3,938,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (1, None, 119)         │       121,975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,090,743 (15.60 MB)

 Trainable params: 4,090,743 (15.60 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:
# Generate the text with default temperature (1.0).
print(generate_text(model, start_string=u"A holdra úgy jutunk el, hogy", temperature=0.4))

A holdra úgy jutunk el, hogy milyen a súlyuk
jóformán nagyobb volt, mint a természet erőinek munkáját - az ember munkáját sem sikerült megint betőlük. De a levegőben lebeg.
Barbicane nyomára az első csákány vágásnál! - mondta az elnök.
- Én meg az utolsó intézkedé-
seket. Úgy rendezkedtek, mint két módszeres gondolkodású Barbicane, akinek más és más domborzatot
alakítottak re. A másik oldalon a Hold verte vissza a nagy T mínusz s, gyalok kony?”-kró területen könnyen levetének előbb helyzetetbe való talpraesett beavatkozásának csúcsaihoz, hogy milyen a Hold vonzóereje egyensúlyban van; erről a pontról 50 000 másodperc, vagyis 83 óra és 20 perc alatt éri el azt a pontot, ahol a fák minden népe díszítettet. Csak egy
történet rejtett ki kell azonban hamarosan megfogalmazott javaslatot. De mi lett türelmetlenül.
- Mi akadálya van annak, hogy kielégítse a tökéletesség hirtelen megszűnésével olyan óriási hőség kell elérnek a hegyeknek a méretek alt a vonalak mivoltát. Ezek a
volt ehhez a most